# Optimierung

In diesem Kapitel wird die Optimierung auf logischer Ebene betrachtet. Es geht mehr um die Daten selbst als um die Speicherabbildung. 

## Wiederholung: Anfragebearbeitung
Zunächst eine kleine Wiederholung zur Anfragebearbeitung in Bezug auf das Grundproblem und den Ablauf.

### Grundproblem
Anfragen in SQL und der Relationalen Algebra sind deklarativ. Solche Anfragen müssen in ausführbare (prozedurale) Form transformiert werden. Also in echte ausführbare Programme. Die Ziele dabei sind ein "QEP" (prozeduraler Query Execution Plan) und Effizienz. Eine Anfrage soll schnell und wenige Ressourcen verbrauchen (CPU, I/O, RAM, Bandbreite). 

### Ablauf der Anfragebearbeitung

<img src="pictures/Ablauf-Anfragebearbeitung.png" alt="Ablauf-Anfragebearbeitung" width="250" style="background-color: white;"/>

1. **Parsing**</br>
    Als erstes wird die Anfrage geparsed und überprüft, ob diese syntaktisch korrekt ist. Danach werden die Elemente semantisch überprüft und ein Parsebaum erstellt. 

2. **Wahl des logischen Anfrageplans**</br>
    Im zweiten Schritt werden exponentiell viele Bäume mit logischen Operatoren erstellt. Darunter wird der optimale Plan ausgewählt, indem vorher logisch, regelbasiert und kostenbasiert optimiert wird. 

3. **Wahl des physischen Anfrageplans**</br>
    Anhand des logischen Plans wird ein ausführbares Programm mit physischen Operatoren erstellt. Das Programm enthält Algorithmen und Scan-Operatoren. Der optimale Plan wird physisch optimiert und dann ausgewählt. Zum Schluss wird der ausgewählte Anfrageplan ausgeführt. 

<img src="pictures/Meme-Finish-SQL-Query.png" alt="Meme-Finish-SQL-Query" width="250" style="background-color: white;"/>

## Ablauf der Anfragebearbeitung 

Die einzelnen Schritte bei der Anfragebearbeitung werden nun genauer betrachtet. 

### Parsing 

<img src="pictures/Ablauf-Anfragebearbeitung_2.png" alt="Ablauf-Anfragebearbeitung_2" width="500" style="background-color: white;"/>

Beginnend beim Parsing wird auf **syntaktische Korrektheit** überprüft. Dafür wird eine Grammatik für einen Teil von SQL betrachtet. (SFW steht für SelectFromWhere): </br>

- Anfragen
    - \<Anfrage> :: = \<SFW>
    - \<Anfrage> :: = ( \<SFW> )
    - die Mengenoperatoren fehlen
- SFWs
    - \<SFW> ::= SELECT \<SelListe> FROM \<FromListe> WHERE \<Bedingung>
    - die Gruppierung, Sortierung etc. fehlen
- Listen
    - \<SelListe> ::= \<Attribut>, \<SelListe>
    - \<SelListe> ::= \<Attribut>
    - \<FromListe> ::= \<Relation>, \<FromListe>
    - \<FromListe> ::= \<Relation>
- Bedingungen
    - \<Bedingung> ::= \<Bedingung> AND \<Bedingung>
    - \<Bedingung> ::= \<Tupel> IN \<Anfrage>
    - \<Bedingung> ::= \<Attribut> = \<Attribut>
    - \<Bedingung> ::= \<Attribut> LIKE \<Muster>
- \<Tupel>, \<Attribut>, \<Relation> und \<Muster> sind nicht durch eine grammatische Regel definiert

Die vollständige Grammatik kann man zum Beispiel [hier]( http://docs.openlinksw.com/virtuoso/GRAMMAR.html) finden. 
</br></br>

Anschließend wird während der Übersetzung die **semantische Korrektheit** überprüft. Beantwortet werden dabei generell Fragen wie:
- Existieren die Relationen und Sichten der FROM Klausel?
- Existieren die Attribute in den genannten Relationen? Sind sie eindeutig?
- Sind die Typen für die Vergleiche korrekt?
- Ist die Aggregation korrekt?   


## Algebraische Transformationsregeln 

<img src="pictures/Ablauf-Anfragebearbeitung_3.png" alt="Ablauf-Anfragebearbeitung_3" width="500" style="background-color: white;"/>

Das Ziel ist es aus dem Parsebaum einen logischen Anfrageplan zu erstellen. Ohne die Semantik der Anfrage zu ändern, soll sie in eine interne Darstellung verwandelt werden. Um die Anfrage effizienter auszuführen, sollen insbesondere kleine Zwischenergebnisse erzielt werden. </br>
Am Ende sollen es äquivalente Ausdrücke sein. Zwei Ausdrücke der relationalen Algebra heißen äquivalent, falls sie die gleichen Operanden (=Relationen) nutzt und für jede mögliche Instanz der Datenbank stets die gleiche Antwortrelation ausgibt. 


### Kommutativität und Assoziativität

Die Gesetze gelten jeweils für Mengen und Multimengen. Die verwendeten Ausdrücke können in beide Richtungen verwendet werden. 

- ? ist kommutativ und assoziativ
    - R ? S = S ? R
    - (R ? S) ? T = R ? (S ? T)
- ∪ ist kommutativ und assoziativ
    - R ∪ S = S ∪ R 
    - (R ∪ S) ∪ T = R ∪ (S ∪ T) 
- ∩ ist kommutativ und assoziativ 
    - R ∩ S = S ∩ R 
    - (R ∩ S) ∩ T = R ∩ (S ∩ T)
- ⨝ ist kommutativ und assoziativ 
    - R ⨝ S = S ⨝ R 
    - (R ⨝ S) ⨝ T = R ⨝ (S ⨝ T)
    

### Weitere Regeln

Für Selektionen und Projektionen gelten die folgenden Regeln: 

- Selektion σ
    - $$σ_{c1 AND c2}(R) = σ_{c1}(σ_{c2}(R))$$
    - $$σ_{c1 OR c2}(R) = σ_{c1}(R) ∪ σ_{c2}(R)$$
        - nicht bei Multimengen
    - $$ σ_{c1}(σ_{c2}(R)) = σ_{c2}(σ_{c1}(R))$$
    - $$ σ_{c}(R Φ S) ≡ (σ_{c}(R)) Φ (σ_{c}(S))$$
        - Φ ∈ {∪, ∩, -, ⨝}
    - $$ σ_{c}(R Φ S) ≡ (σ_{c}(R)) Φ S$$
        - Φ ∈ {∪, ∩, -, ⨝}
        - Falls sich c nur auf die Attribute in R bezieht. 
</br>

- Projektion π
    - $$π_{L}(R ⨝ S) = π_{L}(π_{M}(R) ⨝ π_{N}(S))$$
    - $$π_{L}(R ⨝_{C} S) = π_{L}(π_{M}(R) ⨝_{C} π_{N}(S))$$
    - $$π_{L}(R x S) = π_{L}(π_{M}(R) x π_{N}(S))$$
    - $$π_{L}(σ_{c}(R)) = π_{L}(σ_{c}(π_{M}(R))$$


## Logische Anfragepläne

<img src="pictures/Ablauf-Anfragebearbeitung_4.png" alt="Ablauf-Anfragebearbeitung_4" width="500" style="background-color: white;"/>

<img src="pictures/Anfragebearbeitung-Beispiel-1.png" alt="Anfragebearbeitung-Beispiel-1" width="500" style="background-color: white;"/>

<img src="pictures/Anfragebearbeitung-Beispiel-2.png" alt="Anfragebearbeitung-Beispiel-2" width="500" style="background-color: white;"/>

<img src="pictures/Anfragebearbeitung-Beispiel-3.png" alt="Anfragebearbeitung-Beispiel-3" width="500" style="background-color: white;"/>

<img src="pictures/Ablauf-Anfragebearbeitung_5.png" alt="Ablauf-Anfragebearbeitung_5" width="500" style="background-color: white;"/>

<img src="pictures/TimeCost-Meme.png" alt="TimeCost-Meme" width="500" style="background-color: white;"/>

<img src="pictures/Kostenmodell.png" alt="Kostenmodell" width="500" style="background-color: white;"/>

<img src="pictures/Statistiken-Histogram-of-arrivals.png" alt="Statistiken-Histogram-of-arrivals" width="500" style="background-color: white;"/>

<img src="pictures/Selektivität-schätzen.png" alt="Selektivität-schätzen" width="500" style="background-color: white;"/>

<img src="pictures/Beispiel-Skew-Graph.png" alt="Beispiel-Skew-Graph" width="500" style="background-color: white;"/>

<img src="pictures/George-Kingsley-Zipf.png" alt="George-Kingsley-Zipf" width="500" style="background-color: white;"/>

<img src="pictures/Zipfs-law.png" alt="Zipfs-law" width="500" style="background-color: white;"/>

<img src="pictures/Ablauf-Anfragebearbeitung_6.png" alt="Ablauf-Anfragebearbeitung_6" width="500" style="background-color: white;"/>

<img src="pictures/Verteilungen-Körpergewicht.png" alt="Verteilungen-Körpergewicht" width="500" style="background-color: white;"/>

<img src="pictures/Equi-Width-Histogramme.png" alt="Equi-Width-Histogramme" width="500" style="background-color: white;"/>

<img src="pictures/Equi-Height-Histogramme.png" alt="Equi-Height-Histogramme" width="500" style="background-color: white;"/>

<img src="pictures/Hill-Climbing.png" alt="Hill-Climbing" width="500" style="background-color: white;"/>

<img src="pictures/Richard-Bellman.png" alt="Richard-Bellman" width="500" style="background-color: white;"/>

<img src="pictures/Knapsach-Zweihander.png" alt="Knapsach-Zweihander" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-SaintsChime.png" alt="Knapsack-SaintsChime" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-DragonriderBow.png" alt="Knapsack-DragonriderBow" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-Rapier.png" alt="Knapsack-Rapier" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-DragonCrestShield.png" alt="Knapsack-DragonCrestShield" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-ShortBow.png" alt="Knapsack-ShortBow" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-General.png" alt="Knapsack-General" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-Ausrüstung.png" alt="Knapsack-Ausrüstung" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-Rüstung.png" alt="Knapsack-Rüstung" width="500" style="background-color: white;"/>

<img src="pictures/Knapsack-mit-DP.png" alt="Knapsack-mit-DP" width="500" style="background-color: white;"/>

<img src="pictures/DP-Traveling-Salesman-Problem.png" alt="DP-Traveling-Salesman-Problem" width="500" style="background-color: white;"/>

<img src="pictures/Dynamic-Programming.png" alt="Dynamic-Programming" width="500" style="background-color: white;"/>

<img src="pictures/Anfrageplanung.png" alt="Anfrageplanung" width="500" style="background-color: white;"/>

<img src="pictures/Selinger.png" alt="Selinger" width="500" style="background-color: white;"/>

<img src="pictures/AccessPathSelection-IBM.png" alt="AccessPathSelection-IBM" width="500" style="background-color: white;"/>

<img src="pictures/Dynamische-Programmierung.png" alt="Dynamische-Programmierung" width="500" style="background-color: white;"/>

<img src="pictures/Dynamische-Programmierung_2.png" alt="Dynamische-Programmierung_2" width="500" style="background-color: white;"/>

<img src="pictures/DP-Beispiel.png" alt="DP-Beispiel" width="500" style="background-color: white;"/>

<img src="pictures/DP-Beispiel_2.png" alt="DP-Beispiel_2" width="500" style="background-color: white;"/>

<img src="pictures/DP-Beispiel_3.png" alt="DP-Beispiel_3" width="500" style="background-color: white;"/>

<img src="pictures/DP-Beispiel_4.png" alt="DP-Beispiel_4" width="500" style="background-color: white;"/>

<img src="pictures/DP-Algorithmus.png" alt="DP-Algorithmus" width="500" style="background-color: white;"/>